# Flowers Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to do image classification from scratch on a flowers dataset using the Estimator API.

In [3]:
import os
PROJECT = "qwiklabs-gcp-6ade0c414dd3d823" # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-6ade0c414dd3d823" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-east1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "cnn"

# do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "1.13"  # Tensorflow version

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Input functions to read JPEG images

The key difference between this notebook and [the MNIST one](./mnist_models.ipynb) is in the input function.
In the input function here, we are doing the following:
* Reading JPEG images, rather than 2D integer arrays.
* Reading in batches of batch_size images rather than slicing our in-memory structure to be batch_size images.
* Resizing the images to the expected HEIGHT, WIDTH. Because this is a real-world dataset, the images are of different sizes. We need to preprocess the data to, at the very least, resize them to constant size.

## Run as a Python module

Since we want to run our code on Cloud ML Engine, we've packaged it as a python module.

The `model.py` and `task.py` containing the model code is in <a href="flowersmodel">flowersmodel</a>

**Complete the TODOs in `model.py` before proceeding!**

Once you've completed the TODOs, run it locally for a few steps to test the code.

In [8]:
%%bash
rm -rf flowersmodel.tar.gz flowers_trained
gcloud ml-engine local train \
    --module-name=flowersmodel.task \
    --package-path=${PWD}/flowersmodel \
    -- \
    --output_dir=${PWD}/flowers_trained \
    --train_steps=5 \
    --learning_rate=0.01 \
    --batch_size=2 \
    --model=$MODEL_TYPE \
    --augment \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

W0903 08:24:40.722965 139998767510976 deprecation_wrapper.py:119] From flowersmodel/model.py:23: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0903 08:24:40.723252 139998767510976 deprecation_wrapper.py:119] From flowersmodel/model.py:23: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0903 08:24:40.725013 139998767510976 deprecation_wrapper.py:119] From flowersmodel/model.py:207: The name tf.summary.FileWriterCache is deprecated. Please use tf.compat.v1.summary.FileWriterCache instead.

I0903 08:24:40.725212 139998767510976 run_config.py:528] TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/08_image/labs/flowers_trained', u'--train_steps=5', u'--learning_rate=0.01', u'--batch_size=2', u'--model=cnn', u'--augment', u'--train_data_path=gs://cloud-ml-data/img/fl

Now, let's do it on ML Engine. Note the --model parameter

In [9]:
%%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=flowersmodel.task \
    --package-path=${PWD}/flowersmodel \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=1000 \
    --learning_rate=0.01 \
    --batch_size=40 \
    --model=$MODEL_TYPE \
    --augment \
    --batch_norm \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

gs://qwiklabs-gcp-6ade0c414dd3d823/flowers/trained_cnn us-east1 flowers_cnn_190903_082714
jobId: flowers_cnn_190903_082714
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [flowers_cnn_190903_082714] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe flowers_cnn_190903_082714

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs flowers_cnn_190903_082714


In [11]:
!gcloud ai-platform jobs describe flowers_cnn_190903_082714

createTime: '2019-09-03T08:27:17Z'
etag: hUrp4wyFn3Q=
jobId: flowers_cnn_190903_082714
startTime: '2019-09-03T08:29:18Z'
state: RUNNING
trainingInput:
  args:
  - --output_dir=gs://qwiklabs-gcp-6ade0c414dd3d823/flowers/trained_cnn
  - --train_steps=1000
  - --learning_rate=0.01
  - --batch_size=40
  - --model=cnn
  - --augment
  - --batch_norm
  - --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv
  - --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv
  jobDir: gs://qwiklabs-gcp-6ade0c414dd3d823/flowers/trained_cnn
  packageUris:
  - gs://qwiklabs-gcp-6ade0c414dd3d823/flowers_cnn_190903_082714/0b8616c814e2d3570fd80d85836d8831e595fb709eee2f85046bbc7e1a21a3aa/flowersmodel-0.0.0.tar.gz
  pythonModule: flowersmodel.task
  region: us-east1
  runtimeVersion: '1.13'
  scaleTier: BASIC_GPU
trainingOutput:
  consumedMLUnits: 0.34

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/flowers_cnn_190903_082714?project=qwiklabs-gcp-6ade

## Monitoring training with TensorBoard

Use this cell to launch tensorboard

In [10]:
from google.datalab.ml import TensorBoard
TensorBoard().start("gs://{}/flowers/trained_{}".format(BUCKET, MODEL_TYPE))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

TensorBoard was started successfully with pid 2938. Click here to access it.

2938

In [ ]:
for pid in TensorBoard.list()["pid"]:
    TensorBoard().stop(pid)
    print("Stopped TensorBoard with pid {}".format(pid))

Here are my results:

Model | Accuracy | Time taken | Run time parameters
--- | :---: | ---
cnn with batch-norm | 0.582 | 47 min | 1000 steps, LR=0.01, Batch=40
as above, plus augment | 0.615 | 3 hr | 5000 steps, LR=0.01, Batch=40

What was your accuracy?

## Deploying and predicting with model

Deploy the model:

In [ ]:
%%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

The online prediction service expects images to be base64 encoded as described [here](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#binary_data_in_prediction_input).

In [ ]:
%%bash
IMAGE_URL=gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg

# Copy the image to local disk.
gsutil cp $IMAGE_URL flower.jpg

# Base64 encode and create request message in json format.
python -c 'import base64, sys, json; img = base64.b64encode(open("flower.jpg", "rb").read()).decode(); print(json.dumps({"image_bytes":{"b64": img}}))' &> request.json

Send it to the prediction service

In [ ]:
%%bash
gcloud ml-engine predict \
    --model=flowers \
    --version=${MODEL_TYPE} \
    --json-instances=./request.json

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>